In [1]:
import nengo
import nengo_spa as spa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
np.random.seed(0)

# Define ToH Environment

In [3]:
class TowerOfHanoi:
    def __init__(self, disk_count, vocab, threshold=0.5):
        self.D = vocab.dimensions
        self.pstc = 0.01
        self.disk_count = disk_count
        self.threshold = threshold
        
        self.pegs = [vocab.parse('A'), vocab.parse('B'), vocab.parse('C')]
        self.disks = [vocab.parse('D{}'.format(i)) for i in range(self.disk_count)] + [vocab.parse('NONE')]
        self.reset()
        self.location_dict = {'A':0, 'B':1, 'C':2}  # location_dict[toh.target_peg] -> goal peg
        self.zero = [0]*self.D
        self.vocab = vocab
        
    def reset(self, randomize=True):
        self.location = ['A']*self.disk_count
        #self.location=['A','C','C']
        self.focus_disk = len(self.disks) - 1    # focus disk
        self.largest = self.disk_count - 1
        self.goal = 2                       # goal disk
        self.target_peg = 'C'
        self.move_peg_data = [0] * self.disk_count
        self.goal_peg_data = [0] * self.disk_count
        # constant
        self.target = ['C'] * self.disk_count
        
    def move(self, disk, peg):
        assert self.can_move(disk,peg)
        self.location[disk] = peg
        
    def peg(self, disk):
        return self.location[disk]
        
    def can_move(self, disk, peg):
        assert peg in 'ABC'
        pegs = [self.peg(disk), peg]
        for i in range(disk):
            if self.peg(i) in pegs:
                return False
        return True
        
    def __call__(self, t, x):
        #######
        # Input
        #######
        focus_disk_in = x[:self.D]
        goal_peg_in = x[self.D:2*self.D]
        goal_disk_in = x[2*self.D:3*self.D]
        
        # motor cortex input
        move_peg = x[3*self.D:4*self.D]
        move_disk = x[4*self.D:5*self.D]
        
        ############
        # Processing
        ############
        self.focus_disk = np.argmax(spa.similarity(focus_disk_in, self.disks))
        self.goal_peg_data = spa.similarity(goal_peg_in, self.disks)
        
        ##
        disks = spa.similarity(goal_disk_in, self.disks)
        pegs = self.goal_peg_data
        if np.max(pegs) > self.threshold and np.max(disks) > self.threshold:
            self.goal = np.argmax(disks)
            self.target_peg = 'ABC'[np.argmax(pegs)]
        
        self.move_peg_data = spa.similarity(move_peg, self.pegs)
        ##
        
        ##
        disks = spa.similarity(move_disk, self.disks)
        disk = np.argmax(disks)
        pegs = self.move_peg_data
        peg = 'ABC'[np.argmax(pegs)] # 'ABC' is a char array
        
        if(np.max(pegs) > self.threshold and np.max(disks) > self.threshold):
            if peg != self.peg(disk):
                if self.can_move(disk, peg):
                    self.move(disk, peg)
                    print('Moving D{} to {}'.format(disk, peg))
                else:    
                    print('Cannot move D{} to {}'.format(disk,peg))
        ##
        ########
        # Output
        ########
        
        # define output array
        out = [0]*7*self.D
        out[:self.D] = self.disks[self.largest].v # largest
        out[self.D:2*self.D] = self.disks[self.focus_disk].v # focus_out
        out[2*self.D:3*self.D] = self.vocab.parse(self.peg(self.goal)).v # goal_peg_out
        out[3*self.D:4*self.D] = self.vocab.parse(self.target_peg).v # target_peg
        
        # visual cortex output
        out[4*self.D:5*self.D] = self.disks[self.goal].v # goal_out
        out[5*self.D:6*self.D] = self.vocab.parse(self.target[self.goal]).v # goal_peg_final
        out[6*self.D:7*self.D] = self.zero if self.focus_disk >= self.disk_count else self.vocab.parse(self.peg(self.focus_disk)).v # focus_peg
        
        out_viz = [0]*(3 + len(self.location))
        out_viz[0] = self.focus_disk                        # focus_viz
        out_viz[1] = self.goal                              # goal_viz
        out_viz[2] = self.location_dict[self.target_peg]    # peg_viz
        for idx, loc in enumerate(self.location):
            out_viz[3 + idx] = self.location_dict[loc]      # pos_viz
        out += out_viz
        return out
    
    def __str__(self):
        res = '=============================\nTOH Stats\n============================='
        res += '\n    location: {}'.format(self.location)
        res += '\n    focus_disk: {}'.format(self.focus_disk)
        res += '\n    largest: {}'.format(self.largest)
        res += '\n    goal: {}'.format(self.goal)
        res += '\n    target_peg: {}'.format(self.target_peg)
        res += '\n    move_peg_data: {}'.format(self.move_peg_data)
        res += '\n    goal_peg_data: {}'.format(self.goal_peg_data)
        res += '\n    target: {}'.format(self.target)
        res += '\n============================='
        return res

In [4]:
def toh_node_create(disk_count, vocab):
    D = vocab.dimensions
    toh = TowerOfHanoi(disk_count, vocab)
    with nengo.Network(label='Tower of Hanoi Node') as toh_n:
        toh_n.toh_node = nengo.Node(toh, size_in=5*D, size_out=7*D+6, label='TOH Node')
        
        # connect inputs
        toh_n.focus_disk_in = nengo.Node(size_in=D)
        toh_n.goal_peg_in = nengo.Node(size_in=D)
        toh_n.goal_disk_in = nengo.Node(size_in=D)
        toh_n.move_peg_in = nengo.Node(size_in=D)
        toh_n.move_disk_in = nengo.Node(size_in=D)
        nengo.Connection(toh_n.focus_disk_in, toh_n.toh_node[:D])
        nengo.Connection(toh_n.goal_peg_in, toh_n.toh_node[D:2*D])
        nengo.Connection(toh_n.goal_disk_in, toh_n.toh_node[2*D:3*D])
        nengo.Connection(toh_n.move_peg_in, toh_n.toh_node[3*D:4*D])
        nengo.Connection(toh_n.move_disk_in, toh_n.toh_node[4*D:5*D])
        
        # connect outputs
        toh_n.largest_out = nengo.Node(size_in=D)
        toh_n.focus_disk_out = nengo.Node(size_in=D)
        toh_n.goal_peg_out = nengo.Node(size_in=D)
        toh_n.target_peg_out = nengo.Node(size_in=D)
        toh_n.goal_disk_out = nengo.Node(size_in=D)
        toh_n.goal_peg_final_out = nengo.Node(size_in=D)
        toh_n.focus_peg_out = nengo.Node(size_in=D)
        nengo.Connection(toh_n.toh_node[:D], toh_n.largest_out)
        nengo.Connection(toh_n.toh_node[D:2*D], toh_n.focus_disk_out)
        nengo.Connection(toh_n.toh_node[2*D:3*D], toh_n.goal_peg_out)
        nengo.Connection(toh_n.toh_node[3*D:4*D], toh_n.target_peg_out)
        nengo.Connection(toh_n.toh_node[4*D:5*D], toh_n.goal_disk_out)
        nengo.Connection(toh_n.toh_node[5*D:6*D], toh_n.goal_peg_final_out)
        nengo.Connection(toh_n.toh_node[6*D:7*D], toh_n.focus_peg_out)
        
        # connect visualization
        toh_n.focus_viz = nengo.Node(size_in=1)#, label="focus disk")
        toh_n.goal_viz = nengo.Node(size_in=1)#, label="goal disk")
        toh_n.peg_viz = nengo.Node(size_in=1)#, label="goal peg")
        toh_n.pos_viz = nengo.Node(size_in=3)
        nengo.Connection(toh_n.toh_node[-6], toh_n.focus_viz)
        nengo.Connection(toh_n.toh_node[-5], toh_n.goal_viz)
        nengo.Connection(toh_n.toh_node[-4], toh_n.peg_viz)
        nengo.Connection(toh_n.toh_node[-3:], toh_n.pos_viz)
        
        
    return toh_n

## Create Visualization Hooks

In [5]:
def toh_vis_debug(vocab, disk_count, label='visualization/debug'):
    model = nengo.Network(label)
    with model:
        model.hanoi_node = toh_node_create(disk_count, vocab)
        #hanoi_node = toh_node
        
        ##### Node for visualization #####
        def viz_func(t, x):
            focus_peg = [0]*3
            if x[0] < 3:
                focus_peg[int(x[0])] = 255
            #print("focus_peg: {}".format(focus_peg))
            
            goal_disc = [0]*3
            if x[1] < 3:
                goal_disc[int(x[1])] = 255
            #print("goal_disc: {}".format(goal_disc))
            
            focus_disc = [0]*3
            if x[1] < 3:
                focus_disc[int(x[2])] = 255
            #print("focus_disc: {}".format(focus_disc))

            location = x[3:6]
            viz_func._nengo_html_ = '''
            <svg width="400" height="110">
              <rect x="50" y="0" width="10" height="600" style="fill:rgb(0,0,%i);" />
              <rect x="150" y="0" width="10" height="600" style="fill:rgb(0,0,%i);" />
              <rect x="250" y="0" width="10" height="600" style="fill:rgb(0,0,%i);" />
              
              <rect x="%i" y="40" width="40" height="20" style="fill:rgb(%i,0,%i);" />
              <rect x="%i" y="70" width="70" height="15" style="fill:rgb(%i,0,%i);" />
              <rect x="%i" y="100" width="100" height="10" style="fill:rgb(%i,0,%i);" />
            </svg>
            ''' %(focus_peg[0], focus_peg[1], focus_peg[2],
                 (35+location[0]*100), focus_disc[0], goal_disc[0],
                 (15+location[1]*100), focus_disc[1], goal_disc[1],
                 (location[2]*100), focus_disc[2], goal_disc[2])
            
        model.viz_node = nengo.Node(viz_func, size_in=6, label='viz_node')
        nengo.Connection(model.hanoi_node.focus_viz, model.viz_node[0])
        nengo.Connection(model.hanoi_node.goal_viz, model.viz_node[1])
        nengo.Connection(model.hanoi_node.peg_viz, model.viz_node[2])
        nengo.Connection(model.hanoi_node.pos_viz, model.viz_node[3:6])
        
        # connect all state inputs for visual
        model.move_disk_in_state = spa.State(vocab, dimensions, label='move_disk_in_state')
        model.move_peg_in_state = spa.State(vocab, dimensions, label='move_peg_in_state')
        model.goal_disk_in_state = spa.State(vocab, dimensions, label='goal_disk_in_state')
        model.goal_peg_in_state = spa.State(vocab, dimensions, label='goal_peg_in_state')
        model.focus_disk_in_state = spa.State(vocab, dimensions, label='focus_disk_in_state')
        nengo.Connection(model.move_disk_in_state.output, model.hanoi_node.move_disk_in, synapse=None)
        nengo.Connection(model.move_peg_in_state.output, model.hanoi_node.move_peg_in, synapse=None)
        nengo.Connection(model.goal_disk_in_state.output, model.hanoi_node.goal_disk_in, synapse=None)
        nengo.Connection(model.goal_peg_in_state.output, model.hanoi_node.goal_peg_in, synapse=None)
        nengo.Connection(model.focus_disk_in_state.output, model.hanoi_node.focus_disk_in, synapse=None)
        
        # connect all state outputs for visual
        model.largest_out_state = spa.State(vocab, dimensions, label='largest_out_state')
        model.focus_disk_out_state = spa.State(vocab, dimensions, label='focus_disk_out_state')
        model.goal_peg_out_state = spa.State(vocab, dimensions, label='goal_peg_out_state')
        model.target_peg_out_state = spa.State(vocab, dimensions, label='target_peg_out_state')
        model.goal_disk_out_state = spa.State(vocab, dimensions, label='goal_disk_out_state')
        model.goal_peg_final_out_state = spa.State(vocab, dimensions, label='goal_peg_final_out_state')
        model.focus_peg_out_state = spa.State(vocab, dimensions, label='focus_peg_out_state')
        nengo.Connection(model.hanoi_node.largest_out, model.largest_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.focus_disk_out, model.focus_disk_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.goal_peg_out, model.goal_peg_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.target_peg_out, model.target_peg_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.goal_disk_out, model.goal_disk_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.goal_peg_final_out, model.goal_peg_final_out_state.input, synapse=None)
        nengo.Connection(model.hanoi_node.focus_peg_out, model.focus_peg_out_state.input, synapse=None)
    return model

# Define ToH Agent

In [6]:
model = nengo.Network('TOH')

def toh_agent(vocab_dim):
    model = nengo.Network('ToH Agent')
    with model:
        env = nengo.Node()
        # Table E.1 is used to define the spa states and subnetworks 
        # of the cortical elements for the Tower of Hanoi model
        with nengo.Network('buffer') as model.buffer:
            # used to control the different stages of the problem-solving algorithm
            model.buffer.state = spa.State(vocab=vocab_dim)
            # stores the disk currently being attended to (D0, D1, D2, D3)
            model.buffer.focus = spa.State(vocab=vocab_dim)
            # stores the disk we are trying to move (D0, D1, D2, D3)
            model.buffer.goal = spa.State(vocab=vocab_dim)
            # stores the location we want to move the goal disk to (A, B, C)
            model.buffer.goal_target = spa.State(vocab=vocab_dim)
        
        with nengo.Network('sensory') as model.sensory:
            # automatically contains the location of the focus disk (A, B, C)
            model.sensory.focus_peg = spa.State(vocab=vocab_dim)
            # automatically contains the location of the goal disk (A, B, C)
            model.sensory.goal_current = spa.State(vocab=vocab_dim)
            # automatically contains the final desired location of the goal disk (A, B, C)
            model.sensory.goal_final = spa.State(vocab=vocab_dim)
            # automatically contains the largest visible disk (D3)
            model.sensory.largest = spa.State(vocab=vocab_dim)
            # automcatically contains DONE if the motor action is finished
            model.sensory.motor = spa.State(vocab=vocab_dim)
            
        with nengo.Network('memory') as model.memory:
            # stores an association between mem1 and mem2 in working memory
            model.memory.mem_1 = spa.State(vocab=vocab_dim)
            # stores an association between mem1 and mem2 in working memory
            model.memory.mem_2 = spa.State(vocab=vocab_dim)
            # indicates one element of a pair to attempt to recall from working memory
            model.memory.request = spa.State(vocab=vocab_dim)
            # the vector associated with the currently requested vector
            model.memory.recall = spa.State(vocab=vocab_dim)
            
        with nengo.Network('motor') as model.motor:
            # tells the motor system which disk to move (A, B, C)
            model.motor.move_disk = spa.State(vocab=vocab_dim)
            # tells the motor system where to move the disk to (A, B, C)
            model.motor.move_peg = spa.State(vocab=vocab_dim)
            
        # Table E.2 is used to define the spa rules for the Tower of Hanoi model
        with nengo.Network('TOH Rules') as model.rules:
            with spa.ActionSelection() as model.rules.action_sel:
                spa.ifmax('LookDone',
                    -spa.dot(model.buffer.focus, spa.sym.D0) +
                    spa.dot(model.buffer.goal, model.buffer.focus) + 
                    spa.dot(model.sensory.goal_current, model.buffer.goal_target) + 
                    spa.dot(model.buffer.state, spa.sym.STORE),
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.focus,
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.goal,
                    model.sensory.goal_final >> model.buffer.goal_target)
                spa.ifmax('LookNotDone',
                    -spa.dot(model.buffer.focus, spa.sym.D0) +
                    spa.dot(model.buffer.goal, model.buffer.focus) + 
                    -spa.dot(model.sensory.goal_current, model.buffer.goal_target) + 
                    spa.dot(model.buffer.state, spa.sym.STORE),
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.focus)
                spa.ifmax('InTheWay1', 
                    -spa.dot(model.buffer.focus, model.buffer.goal) + 
                    spa.dot(model.sensory.focus_peg, model.sensory.goal_current) + 
                    -spa.dot(model.sensory.focus_peg, model.buffer.goal_target) + 
                    -spa.dot(model.buffer.state, spa.sym.STORE),
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.focus)
                spa.ifmax('InTheWay2',
                    -spa.dot(model.buffer.focus, model.buffer.goal) + 
                    -spa.dot(model.sensory.focus_peg, model.sensory.goal_current) + 
                    spa.dot(model.sensory.focus_peg, model.buffer.goal_target) + 
                    -spa.dot(model.buffer.state, spa.sym.STORE),
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.focus)
                spa.ifmax('NotInTheWay',
                    -spa.dot(model.buffer.focus, model.buffer.goal) + 
                    -spa.dot(model.sensory.focus_peg, model.sensory.goal_current) + 
                    -spa.dot(model.sensory.focus_peg, model.buffer.goal_target) + 
                    -spa.dot(model.buffer.focus, spa.sym.D0),
                    model.buffer.goal * spa.sym.NEXT >> model.buffer.focus)
                
                spa.ifmax('MoveD0',
                    spa.dot(model.buffer.focus, spa.sym.D0) + 
                    spa.dot(model.buffer.goal, spa.sym.D0) + 
                    -spa.dot(model.sensory.goal_current, model.buffer.goal_target),
                    spa.sym.D0 >> model.motor.move_disk,
                    model.buffer.goal_target >> model.motor.move_peg)
                spa.ifmax('MoveGoal',
                    spa.dot(model.buffer.focus, spa.sym.D0) + 
                    -spa.dot(model.buffer.goal, spa.sym.D0) + 
                    -spa.dot(model.sensory.focus_peg, model.buffer.goal_target) + 
                    -spa.dot(model.buffer.goal_target, model.sensory.goal_current) + 
                    -spa.dot(model.sensory.focus_peg, model.sensory.goal_current),
                    model.buffer.goal >> model.motor.move_disk,
                    model.buffer.goal_target >> model.motor.move_peg)
                spa.ifmax('MoveDone',
                    spa.dot(model.sensory.motor, spa.sym.DONE) + 
                    -spa.dot(model.buffer.goal, model.sensory.largest) + 
                    -spa.dot(model.buffer.state, spa.sym.RECALL),
                    spa.sym.RECALL >> model.buffer.state,
                    model.buffer.goal * ~spa.sym.NEXT >> model.buffer.goal)
                spa.ifmax('MoveDone2',
                    spa.dot(model.sensory.motor, spa.sym.DONE) + 
                    spa.dot(model.buffer.goal, model.sensory.largest) + 
                    -spa.dot(model.buffer.state, spa.sym.RECALL),
                    model.sensory.largest * ~spa.sym.NEXT >> model.buffer.focus,
                    model.sensory.largest * ~spa.sym.NEXT >> model.buffer.goal,
                    model.sensory.goal_final >> model.buffer.goal_target,
                    spa.sym.HANOI >> model.buffer.state)
                    
                spa.ifmax('Store',
                    spa.dot(model.buffer.state, spa.sym.STORE) + 
                    -spa.dot(model.memory.recall, model.buffer.goal_target),
                    model.buffer.goal >> model.memory.mem_1,
                    model.buffer.goal_target >> model.memory.mem_2,
                    model.buffer.goal >> model.memory.request)
                spa.ifmax('StoreDone',
                    spa.dot(model.buffer.state, spa.sym.STORE) + 
                    spa.dot(model.memory.recall, model.buffer.goal_target),
                    spa.sym.FIND >> model.buffer.state)
                
                spa.ifmax('FindFree1',
                    spa.dot(model.buffer.state, spa.sym.FIND) + 
                    -spa.dot(model.buffer.focus, model.buffer.goal) + 
                    spa.dot(model.sensory.focus_peg, model.sensory.goal_current) + 
                    -spa.dot(model.sensory.focus_peg, model.buffer.goal_target),
                    spa.sym.A + spa.sym.B + spa.sym.C - model.sensory.focus_peg - model.buffer.goal_target >> model.buffer.goal_target,
                    model.buffer.focus >> model.buffer.goal,
                    spa.sym.HANOI >> model.buffer.state)
                spa.ifmax('FindFree2',
                    spa.dot(model.buffer.state, spa.sym.FIND) + 
                    -spa.dot(model.buffer.focus, model.buffer.goal) + 
                    -spa.dot(model.sensory.focus_peg, model.sensory.goal_current) + 
                    spa.dot(model.sensory.focus_peg, model.buffer.goal_target),
                    spa.sym.A + spa.sym.B + spa.sym.C - model.sensory.goal_current - model.buffer.goal_target >> model.buffer.goal_target,
                    model.buffer.focus >> model.buffer.goal,
                    spa.sym.HANOI >> model.buffer.state)
                
                spa.ifmax('Recall',
                    spa.dot(model.buffer.state, spa.sym.RECALL) + 
                    -spa.dot(model.memory.recall, spa.sym.A + spa.sym.B + spa.sym.C),
                    model.buffer.goal >> model.memory.request)
                spa.ifmax('RecallDo',
                    spa.dot(model.buffer.state, spa.sym.RECALL) + 
                    spa.dot(model.memory.recall, spa.sym.A + spa.sym.B + spa.sym.C) +
                    -spa.dot(model.memory.recall, model.sensory.goal_current),
                    spa.sym.HANOI >> model.buffer.state,
                    model.buffer.goal >> model.buffer.focus,
                    4 * model.memory.recall >> model.buffer.goal_target)
                spa.ifmax('RecallNext',
                    spa.dot(model.buffer.state, spa.sym.RECALL) + 
                    spa.dot(model.memory.recall, spa.sym.A + spa.sym.B + spa.sym.C) +
                    spa.dot(model.memory.recall, model.sensory.goal_current),
                    spa.sym.HANOI >> model.buffer.state,
                    model.buffer.goal * ~spa.sym.NEXT >> model.buffer.goal,
                    model.buffer.goal >> model.memory.request)
    return model

# Setup Simulation

In [7]:
disk_count = 1
dimensions = 32
vocab = spa.Vocabulary(dimensions)
vocab.populate('A; B; C; D0; D1; D2; D3; STORE; NEXT; DONE; RECALL; FIND; HANOI; NONE')

/home/fandav/.conda/envs/flight_test/lib/python3.5/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=32, M=9, similarity=0.10)
  len(self._key2idx), best_sim))
/home/fandav/.conda/envs/flight_test/lib/python3.5/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=32, M=12, similarity=0.14)
  len(self._key2idx), best_sim))
/home/fandav/.conda/envs/flight_test/lib/python3.5/site-packages/nengo_spa/vocabulary.py:174: UserWarning: Could not create a semantic pointer with max_similarity=0.10 (D=32, M=13, similarity=0.14)
  len(self._key2idx), best_sim))


In [8]:
with model:
    
    agent = toh_agent(vocab)
    vis_network = toh_vis_debug(vocab, disk_count, 'Tower of Hanoi')
    
    # attach environment to sensory
    vis_network.focus_peg_out_state >> agent.sensory.focus_peg
    vis_network.goal_peg_out_state >> agent.sensory.goal_current
    vis_network.target_peg_out_state >> agent.sensory.goal_final
    #vis_network.target_peg_out_state >> agent.sensory.goal_current
    #vis_network.goal_peg_out_state >> agent.sensory.goal_final
    vis_network.largest_out_state >> agent.sensory.largest
    
    # attach motor to environment
    agent.buffer.focus >> vis_network.focus_disk_in_state
    agent.motor.move_disk >> vis_network.move_disk_in_state
    agent.motor.move_peg >> vis_network.move_peg_in_state

In [9]:
# agent probes
with model:
    # probe sensory
    p_sensory_focus_peg = nengo.Probe(agent.sensory.focus_peg.output, synapse=0.01)
    p_sensory_goal_current = nengo.Probe(agent.sensory.goal_current.output, synapse=0.01)
    p_sensory_goal_final= nengo.Probe(agent.sensory.goal_final.output, synapse=0.01)
    p_sensory_largest = nengo.Probe(agent.sensory.largest.output, synapse=0.01)
    p_sensory_motor = nengo.Probe(agent.sensory.motor.output, synapse=0.01)
    
    # probe motor
    #p_buffer_focus = nengo.Probe(agent.buffer.focus.output, synapse=0.01)
    p_motor_move_disk = nengo.Probe(agent.motor.move_disk.output, synapse=0.01)
    p_motor_move_peg = nengo.Probe(agent.motor.move_peg.output, synapse=0.01)

    # probe action selection
    p_selected_actions = nengo.Probe(agent.rules.action_sel.thalamus.output, synapse=0.01)
    p_utility = nengo.Probe(agent.rules.action_sel.bg.input, synapse=0.01)

In [10]:
# environment probes
with model:
    # environment output to agent input
    p_env_focus_peg_out = nengo.Probe(vis_network.focus_peg_out_state.output, synapse=0.01)
    p_env_target_peg_out = nengo.Probe(vis_network.target_peg_out_state.output, synapse=0.01)
    p_env_goal_peg_out = nengo.Probe(vis_network.goal_peg_out_state.output, synapse=0.01)
    p_env_largest_disk_out = nengo.Probe(vis_network.largest_out_state.output, synapse=0.01)
    
    # environment input from agent output
    p_env_focus_disk_in = nengo.Probe(vis_network.focus_disk_in_state.output, synapse=0.01)
    p_env_move_disk_in = nengo.Probe(vis_network.move_disk_in_state.output, synapse=0.01)
    p_env_move_peg_in = nengo.Probe(vis_network.move_peg_in_state.output, synapse=0.01)

In [11]:
import nengo_ocl
sim = nengo_ocl.Simulator(model)

#sim = nengo.Simulator(model)

No context argument was provided to nengo_ocl.Simulator
Calling pyopencl.create_some_context() for you now:


In [12]:
# run simulator
with sim:
    sim.run(1.)

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

ValueError: cannot reshape array of size 228 into shape (230,1)

In [ ]:
vocab_colors = [np.random.rand(3,) for color in vocab.keys()]

### Agent Probes

In [ ]:
# plot probes for agent
pegs_vocab = vocab.create_subset(['A', 'B', 'C'])
disks_vocab = vocab.create_subset(['D0', 'D1', 'D2', 'D3'])
done_vocab = vocab.create_subset(['DONE'])

plt.figure(figsize=(16,10))

# sensory plots
plt.subplot(2,2,1)
plt.plot(sim.trange(), spa.similarity(sim.data[p_sensory_focus_peg], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Focus Peg - Location of Focus Disk (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,2)
plt.plot(sim.trange(), spa.similarity(sim.data[p_sensory_goal_current], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Goal Current - Location of Goal Disk (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,3)
plt.plot(sim.trange(), spa.similarity(sim.data[p_sensory_goal_final], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Goal Final - Final Desired Location of Goal Disk (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,4)
plt.plot(sim.trange(), spa.similarity(sim.data[p_sensory_largest], disks_vocab))
plt.xlabel('Time (s)')
plt.title('Largest - Largest Visible Disk (Disk)')
plt.legend(disks_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.figure(figsize=(16,16))

# motor plots
plt.subplot(3,1,1)
plt.plot(sim.trange(), spa.similarity(sim.data[p_motor_move_disk], vocab))
plt.xlabel('Time (s)')
plt.title('Move Disk - Which Disk to Move (Disk)')
plt.legend(vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(3,1,2)
#[plt.plot(sim.trange(), data, color=color) for color, data in zip(vocab_colors, spa.similarity(sim.data[p_sensory_goal_current], pegs_vocab).T)]
plt.plot(sim.trange(), spa.similarity(sim.data[p_motor_move_peg], vocab))
plt.xlabel('Time (s)')
plt.title('Move Peg - Where to Move Disk (Peg)')
plt.legend(vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(3,1,3)
#[plt.plot(sim.trange(), data, color=color) for color, data in zip(vocab_colors, spa.similarity(sim.data[p_sensory_goal_current], pegs_vocab).T)]
plt.plot(sim.trange(), spa.similarity(sim.data[p_sensory_motor], done_vocab))
plt.xlabel('Time (s)')
plt.title('Motor - DONE When Motor Action Finished (DONE)')
plt.legend(done_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])


In [ ]:
as_colors = [np.random.rand(3,) for color in agent.rules.action_sel.keys()]

In [ ]:
keys = [key for key in agent.rules.action_sel.keys()]

# sort by most similar actions taken
get_most_similar_idx = lambda x: np.flip(np.argsort(np.sum(x, axis=0)))

most_similar_utility_idx = get_most_similar_idx(sim.data[p_utility])
most_similar_selected_actions_idx = get_most_similar_idx(sim.data[p_selected_actions])

most_similar_utility = [keys[idx] for idx in most_similar_utility_idx]
most_similar_selected_actions = [keys[idx] for idx in most_similar_selected_actions_idx]

data = {'Utility': most_similar_utility,
        'Selected Actions': most_similar_selected_actions}

df = pd.DataFrame(data, columns=['Utility', 'Selected Actions'])
display(df)

In [ ]:
# plot action selection probes of agent

plt.figure(figsize=(16,13))
plt.subplot(2,1,1)
#plt.plot(sim.trange(), sim.data[p_utility])
[plt.plot(sim.trange(), data, color=color) for color, data in zip(as_colors, sim.data[p_utility].T)]
plt.legend(tuple(agent.rules.action_sel.keys()), loc='best')
plt.ylabel('Utility')
plt.xlabel("Time (s)")
plt.title('Utility Value')

#plt.figure(figsize=(16,8))
plt.subplot(2,1,2)
#plt.plot(sim.trange(), sim.data[p_selected_actions])
[plt.plot(sim.trange(), data, color=color) for color, data in zip(as_colors, sim.data[p_selected_actions].T)]
plt.legend(tuple(agent.rules.action_sel.keys()), loc='best')
plt.ylabel('Selected Action')
plt.xlabel("Time (s)")
plt.title('Selected Action')

### Environment Probes

In [ ]:
# plot probes of ToH environment
pegs_vocab = vocab.create_subset(['A', 'B', 'C'])
disks_vocab = vocab.create_subset(['D0', 'D1', 'D2', 'D3'])
done_vocab = vocab.create_subset(['DONE'])

plt.figure(figsize=(16,10))

# env outputs
plt.subplot(2,2,1)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_focus_peg_out], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Env Focus Peg Out - Location of Focus Disk (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,2)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_target_peg_out], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Env Target Peg Out (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,3)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_goal_peg_out], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Env Goal Peg Out (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(2,2,4)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_largest_disk_out], disks_vocab))
plt.xlabel('Time (s)')
plt.title('Env Largest Disk Out - Largest Visible Disk (Disk)')
plt.legend(disks_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.suptitle('Environment Outputs')


plt.figure(figsize=(16,20))

# env inputs
plt.subplot(3,2,1)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_focus_disk_in], disks_vocab))
plt.xlabel('Time (s)')
plt.title('Env Focus Disk In (Disk)')
plt.legend(disks_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(3,2,2)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_move_disk_in], disks_vocab))
plt.xlabel('Time (s)')
plt.title('Env Move Disk Input - (Disk)')
plt.legend(disks_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

plt.subplot(3,2,3)
plt.plot(sim.trange(), spa.similarity(sim.data[p_env_move_peg_in], pegs_vocab))
plt.xlabel('Time (s)')
plt.title('Env Move Peg In (Peg)')
plt.legend(pegs_vocab.keys(), loc='best')
plt.ylim([-.25, 1.25])

Hmm. We’re having trouble finding that site.

We can’t connect to the server at www.np.reverse.

If that address is correct, here are three other things you can try:

    Try again later.
    Check your network connection.
    If you are connected but behind a firewall, check that Firefox has permission to access the Web.


plt.suptitle('Environment Inputs')